# Backpropogation
- Training a network  (backpropagation) consists of:
  - Initializing weights at “random”.
  - Compute the network forward (forward pass)
  - Reduce loss by updating weights in opposite direction of gradient of the loss function.
  - Repeat the process until an optimized set of weights are calculated.

<img src="img/backprop.png" height="600" width="600">

# Gradient Descent
<img src="img/sgd2d.png" width="480" height="480">

# Loss Function
<img src="img/loss.png" height="480" width="480">

# Non-Convex Optimization
<img src="img/nonconvex.png" height="720" width="720">

# Training a network
- Define the network
- Initialize the network with with random/pre-trained weights
- Choose a loss function
- Choose an optimizer
- Prepare Dataset
- Run back propogation algorithm.
- Evaluate the output

In [5]:
import mxnet as mx
from mxnet import gluon, nd, autograd
import numpy as np
ctx = mx.cpu()

## Define the network

In [6]:
net = gluon.nn.Sequential()

with net.name_scope(): #Returns a name space object managing a child :py:class:`Block` and parameter names.
    net.add(gluon.nn.Dense(units=128, activation='relu'))
    net.add(gluon.nn.Dense(units=64, activation='relu'))
    net.add(gluon.nn.Dense(units=10))

## Initialize the network

In [7]:
net.initialize(mx.init.Xavier(magnitude=2.24), force_reinit=True, ctx=ctx)

## Choose a loss function

In [8]:
loss_fn = gluon.loss.SoftmaxCrossEntropyLoss()

## Choose an Optimizer

In [9]:
trainer = gluon.Trainer(params=net.collect_params(), 
                        optimizer='sgd', 
                        optimizer_params={"learning_rate":0.01, "momentum": .9, "wd":.1})

## Prepare dataset

In [10]:
batch_size = 128
def transform(data, label):
    return (data.astype(np.float32)/255, label.astype(np.float32))

train_dataset = gluon.data.vision.MNIST(train=True, transform=transform)
val_dataset = gluon.data.vision.MNIST(train=False, transform=transform)

train_data_loader = gluon.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_data_loader = gluon.data.DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

(train_dataset[0][0].shape, train_dataset[0][1])


((28, 28, 1), 5.0)

In [11]:
import matplotlib.pyplot as plt
text_labels = [
    'zero', 'one', 'two', 'three', 'four',
    'five', 'six', 'seven', 'eight', 'nine'
]
X, y = train_dataset[0:6]
_, figs = plt.subplots(1, X.shape[0], figsize=(15, 15))
for f,x,yi in zip(figs, X,y):
    # 3D->2D by removing the last channel dim
    f.imshow(x.reshape((28,28)).asnumpy())
    ax = f.axes
    ax.set_title(text_labels[int(yi)])
    ax.title.set_fontsize(20)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()


<Figure size 1500x1500 with 6 Axes>

In [12]:
X.shape

(6, 28, 28, 1)

In [13]:
(train_dataset[0][0].shape, train_dataset[0][1])

((28, 28, 1), 5.0)

In [14]:
for data, label in train_data_loader:
    print(data.shape, label.shape)
    break

(128, 28, 28, 1) (128,)


## Run back propogation algorithm

In [15]:
epochs = 1
for e in range(epochs):
    for i, (data, label) in enumerate(train_data_loader):
        with autograd.record():
            data = data.as_in_context(ctx)
            label = label.as_in_context(ctx)
            outputs = net(data)
            loss = loss_fn(outputs, label)
        loss.backward()
        trainer.step(batch_size)
    print("EPOC: [{}]: ; train loss: {}".format(e, loss.mean().asscalar()))
        

EPOC: [0]: ; train loss: 0.9659042954444885


## Evaluate the Output
- In order to evaluate the putput we need to compare performance of the algorithm on training and evaluate

In [21]:
def evaluate_results(data_loader, network):
    metric = mx.metric.Accuracy()
    
    metric.reset()
    for i, (data, label) in enumerate(data_loader):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        outputs = network(data)
        predictions = nd.argmax(outputs, axis=1)
        metric.update(preds=predictions, labels=label)
    return metric


In [22]:
epochs = 5
for e in range(epochs):
    for i, (data, label) in enumerate(train_data_loader):
        with autograd.record():
            data = data.as_in_context(ctx)
            label = label.as_in_context(ctx)
            outputs = net(data)
            loss = loss_fn(outputs, label)
        loss.backward()
        trainer.step(batch_size)
    train_acc = evaluate_results(train_data_loader, net).get()[1]
    val_acc = evaluate_results(val_data_loader, net).get()[1]
    net.save_parameters('net.params')
    print("EPOCH: [{}]: ; train loss: {}; train_acc: {}; val_acc: {}".format(e, loss.mean().asscalar(), train_acc, val_acc))

EPOCH: [0]: ; train loss: 0.8281387686729431; train_acc: 0.8087166666666666; val_acc: 0.817
EPOCH: [1]: ; train loss: 0.7974648475646973; train_acc: 0.8083833333333333; val_acc: 0.8146
EPOCH: [2]: ; train loss: 0.9794893264770508; train_acc: 0.8111666666666667; val_acc: 0.8178
EPOCH: [3]: ; train loss: 0.768927276134491; train_acc: 0.8066; val_acc: 0.8135
EPOCH: [4]: ; train loss: 0.8989737033843994; train_acc: 0.8082666666666667; val_acc: 0.8135


### Multiple Metrics
- Often we would like to measure several metrics
- An exaple is that correlation and MSE are combined for time-series

In [23]:
def evaluate_results(data_loader, network):
    metric_list = [mx.metric.MSE(), mx.metric.Accuracy(), mx.metric.PearsonCorrelation()]
    metrics = mx.metric.CompositeEvalMetric()
    for m in metric_list:
        metrics.add(m)
    
    metrics.reset()
    for i, (data, label) in enumerate(data_loader):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        outputs = network(data)
        predictions = nd.argmax(outputs, axis=1)
        metrics.update(preds=predictions, labels=label)
    return metrics


In [24]:
epochs = 1
for e in range(epochs):
    for i, (data, label) in enumerate(train_data_loader):
        with autograd.record():
            data = data.as_in_context(ctx)
            label = label.as_in_context(ctx)
            outputs = net(data)
            loss = loss_fn(outputs, label)
        loss.backward()
        trainer.step(batch_size)
    train_acc = evaluate_results(train_data_loader, net)
    val_acc = evaluate_results(val_data_loader, net)
    
    print("EPOCH: [{}]:\ntrain loss: {}".format(e, loss.mean().asscalar()))
    print("training metrics: {}".format(train_acc.get_name_value()))
    print("validation metrics: {}".format(val_acc.get_name_value()))
    print('------------------------------------------------------------------------------------------------------------------------')


EPOCH: [0]:
train loss: 0.9224110245704651
training metrics: [('mse', 3.056719749466951), ('accuracy', 0.8114166666666667), ('pearsonr', 0.8213967369061631)]
validation metrics: [('mse', 2.828223892405063), ('accuracy', 0.8202), ('pearsonr', 0.8348816169997487)]
------------------------------------------------------------------------------------------------------------------------


# Challenge
- Try to rewrite the training code, using c conolutional network form the previous lab
- beware that `gluon.nn.Conv2d()` supports NCHW’ and ‘NHWC’ layout for now. ‘N’, ‘C’, ‘H’, ‘W’ stands for batch, channel, height, and width dimensions respectively. Convolution is applied on the ‘H’ and ‘W’ dimensions. 
- We need to use `nd.transpose()` in order to change the layout

In [31]:
def transform(data, label):
    return nd.transpose(data.astype(np.float32), (2,0,1))/255, label.astype(np.float32)

train_dataset_conv = gluon.data.vision.MNIST(train=True, transform=transform)
(("NEW: ", train_dataset_conv[0][0].shape, train_dataset_conv[0][1]),
("OLD: ", train_dataset[0][0].shape, train_dataset[0][1]))

(('NEW: ', (1, 28, 28), 5.0), ('OLD: ', (28, 28, 1), 5.0))